<a href="https://colab.research.google.com/github/curiasuncc/4106-Real-time-ML/blob/main/Hw3_Q1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

libraries

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
#torch.manual_seed(123)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [ ]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [ ]:
#model = Net()

#numel_list = [p.numel() for p in model.parameters()]
#sum(numel_list), numel_list

In [ ]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1,device=device)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1,device=device)

        self.conv3=nn.Conv2d(16, 16, kernel_size=3, padding=1,device=device)

        self.fc1 = nn.Linear(4 * 4 * 16, 32,device=device)
        self.fc2 = nn.Linear(32, 10,device=device)
        #self.fc3 = nn.Linear(16,10,device=device)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)

        out = F.max_pool2d(torch.relu(self.conv3(out)), 2)

        out = out.view(-1, 4 * 4 * 16)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


In [ ]:
#model = Net()
#model(img.unsqueeze(0))

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [ ]:
#CUDA_LAUNCH_BLOCKING=1

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model = Net().to(device=device)  # <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-03-29 20:08:23.714152 Epoch 1, Training loss 2.1951829376428025
2022-03-29 20:10:57.365567 Epoch 10, Training loss 1.1823122132464747
2022-03-29 20:13:45.489661 Epoch 20, Training loss 0.9125960678853038
2022-03-29 20:16:38.168900 Epoch 30, Training loss 0.8037290882957561
2022-03-29 20:19:31.389770 Epoch 40, Training loss 0.73510301639052
2022-03-29 20:22:22.250309 Epoch 50, Training loss 0.6896011519538777
2022-03-29 20:25:15.177921 Epoch 60, Training loss 0.6529423982819633
2022-03-29 20:28:08.156812 Epoch 70, Training loss 0.6240286410159772
2022-03-29 20:31:01.796483 Epoch 80, Training loss 0.6000068468587173
2022-03-29 20:33:55.930832 Epoch 90, Training loss 0.578468315436712
2022-03-29 20:36:49.809258 Epoch 100, Training loss 0.5585739258724405
2022-03-29 20:39:43.463890 Epoch 110, Training loss 0.5434207306112475
2022-03-29 20:42:35.999515 Epoch 120, Training loss 0.5271323822876987
2022-03-29 20:45:28.188139 Epoch 130, Training loss 0.5126166265372121
2022-03-29 20:48:16.

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

all_acc_dict["baseline"] = validate(model, train_loader, val_loader)

Accuracy train: 0.84
Accuracy val: 0.66
